In [1]:
Pkg.build("PyPlot")
using PyPlot

INFO: Building PyCall
INFO: PyCall is using /usr/bin/python (Python 2.7.6) at /usr/bin/python, libpython = libpython2.7
INFO: Loading help data...


LoadError: ReverseDiffSource not found
while loading In[1], in expression starting on line 3

In [2]:
Pkg.add("ReverseDiffSource")
using ReverseDiffSource

INFO: Cloning cache of ReverseDiffSource from git://github.com/JuliaDiff/ReverseDiffSource.jl.git
INFO: Installing ReverseDiffSource v0.1.4
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of ReverseDiffSource
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [3]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating cache of Convex...
INFO: Updating cache of JSON...
INFO: Updating Toms566...
INFO: Computing changes...
INFO: Cloning cache of DataStructures from git://github.com/JuliaLang/DataStructures.jl.git
INFO: Upgrading ColorTypes: v0.1.6 => v0.1.7
INFO: Upgrading Compat: v0.7.4 => v0.7.6
INFO: Upgrading Convex: v0.0.6 => v0.1.0
INFO: Installing DataStructures v0.3.13
INFO: Upgrading FixedPointNumbers: v0.0.11 => v0.0.12
INFO: Upgrading JSON: v0.4.6 => v0.5.0
INFO: Upgrading MathProgBase: v0.3.17 => v0.3.18
INFO: Upgrading Optim: v0.4.3 => v0.4.4


In [13]:
download("http://www.ats.ucla.edu/stat/data/binary.csv", "binary.csv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5489  100  5489    0     0  20607      0 --:--:-- --:--:-- --:--:-- 20635


"binary.csv"

In [23]:
data = convert(Array{Float64,2},readcsv("binary.csv")[2:end,:][:,1:4]);

In [24]:
y = data[:,1];
u = data[:,2:4];
o = ones(size(u,1));
u = [u o];
m = size(y,1);
n = 4;
eta = 10^-8.0;

In [35]:
function obj(x)
    return (p.obj(x),p.grd(x),p.hes(x))
end

function newtmin(obj, x0; maxIts = 1000, opTol = 1e-6)
    
    its = 0
    (f0,g0,h0) = obj(x0)
    x = x0
    (f,g,h) = obj(x)
    
    while norm(g,2) > 1e-8 && its < maxIts
       
        E = eigfact(h)
        V = E[:vectors]
        Lam = diagm(max(E[:values], 1e-2))
        d = -V*inv(Lam)*transpose(V)*g
        
        alpha = 1
        mu  = 1e-2
        (fp,gp,hp) = obj(x+alpha*d)
        while fp > f + alpha*mu*dot(g,d)
            alpha = alpha/2
            (fp,gp,hp) = obj(x+alpha*d)
        end
        
        x = x + alpha*d
        (f,g,h) = obj(x)
        its = its + 1
        
    end
    
   if norm(g,2) < opTol && its < maxIts
        println("Way 2 go, homeskillet.")
    else
        println("Aw, shucks")
    end
    
    return (x, f , norm(g,2), its)
    
end

newtmin (generic function with 1 method)

In [25]:
function L(a)
    valL = 0
    for i=1:m
        valL = valL -(y[i]*a[1]*u[i,1] + y[i]*a[2]*u[i,2] + y[i]*a[3]*u[i,3] + y[i]*a[4]*u[i,4]) + log(1 + exp(a[1]*u[i,1]+a[2]*u[i,2]+a[3]*u[i,3]+a[4]*u[i,4]))
    end
    return valL
end

L (generic function with 1 method)

In [26]:
diffL = rdiff( L, (ones(n),), order = 2)

(anonymous function)

In [27]:
(f,g,H) = diffL(.5*ones(n))

(79193.41499999998,[156480.0,912.83,721.0,273.0],
4x4 Array{Float64,2}:
      3.3656e7  157145.0    104700.0   43760.0 
 157145.0          741.543     494.29    204.68
 104700.0          494.29      391.0     137.0 
  43760.0          204.68      137.0      57.0 )

In [36]:
(x,f,ng,its) = newtmin(diffL,.5*ones(4))

Way 2 go, homeskillet.


([0.00229396,0.777014,-0.560031,-3.44955],229.7208825156379,1.0643987847804553e-11,50)